In [25]:
# importing packages
import pandas as pd
import re
from bs4 import BeautifulSoup
import requests 
from requests import get
from warnings import warn

In [26]:
def get_urls(raw_url,pages_to_extract,page_inc):
    return [raw_url+'&start='+str((x)*(page_inc)) for x in range(pages_to_extract)]

In [27]:
def get_job_soup(url):
    from requests import get
    source = requests.get(url).text
    soup = BeautifulSoup(source,'lxml')
    #jobs = soup.find('ul', class_ = 'jobs-search__results-list')
    return soup

In [28]:
def get_job_fields(soup):
    job_container_jobs = soup.find_all('a', class_ = 'base-card__full-link')
    job_names = [x.span.text.strip() for x in job_container_jobs]
    job_urls = [x['href'] for x in job_container_jobs]
    job_id = [re.findall(r'(?!-)([0-9]*)(?=\?)',x)[0] for x in job_urls]
    
    job_container_company = soup.find_all('a', class_ = 'hidden-nested-link')
    company_names = [x.text.strip() for x in job_container_company]
    company_urls = [x['href'] for x in job_container_company]
    
    job_info = soup.find_all('div', class_ = 'base-search-card__metadata')
    company_benefits = [str(x) if str(x) == 'None' else x.text.strip() for x in [x.find('span', class_ = 'result-benefits__text') for x in job_info]]
    date_posted = [x.time['datetime'] for x in job_info]
    salary = [str(x) if str(x) == 'None' else str(x.text.strip()) for x in [x.find('span', class_ = 'job-search-card__salary-info') for x in job_info]]
    job_location = [str(x) if str(x) == 'None' else x.text.strip() for x in [x.find('span', class_ = 'job-search-card__location') for x in job_info]]
    job_snippet = [str(x) if str(x) == 'None' else x.text.strip() for x in [x.find('p', class_ = 'job-search-card__snippet') for x in job_info]]
    easy_apply = [str(x) if str(x) == 'None' else 'Easy Apply Enabled' for x in [x.find('span', class_ = 'job-search-card__easy-apply-label') for x in job_info]]
    
    return [job_id,job_names,company_names,date_posted,salary,job_location,easy_apply,company_benefits,job_urls,company_urls,job_snippet]

In [29]:
raw_url = 'https://www.linkedin.com/jobs/search/?geoId=101165590&keywords=data%20analyst&location=United%20Kingdom'
pages_to_extract = 40
page_inc = 25

urls = get_urls(raw_url,pages_to_extract,page_inc)
soups = [get_job_soup(url) for url in urls]
job_fields = [get_job_fields(soup) for soup in soups]

In [30]:
key0 = []
key1 = []
key2 = []
key3 = []
key4 = []
key5 = []
key6 = []
key7 = []
key8 = []
key9 = []
key10 = []

for i in range(pages_to_extract):
    key0.extend(job_fields[i][0])
    key1.extend(job_fields[i][1])
    key2.extend(job_fields[i][2])
    key3.extend(job_fields[i][3])
    key4.extend(job_fields[i][4])
    key5.extend(job_fields[i][5])
    key6.extend(job_fields[i][6])
    key7.extend(job_fields[i][7])
    key8.extend(job_fields[i][8])
    key9.extend(job_fields[i][9])
    key10.extend(job_fields[i][10])
    
link_jobs = pd.DataFrame(data=zip(key0,
                                  key1,
                                  key2,
                                  key3,
                                  key4,
                                  key5,
                                  key6,
                                  key7,
                                  key8,
                                  key9,
                                  key10),
              columns=['Linkedin Job ID','Job Title','Company','Date Posted','Salary',
                       'Location','Easy Apply','Hiring status','Job Link','Company Profile','Job Desc'])

link_jobs_unq = link_jobs.drop_duplicates(subset='Linkedin Job ID', keep='first', inplace=False).sort_values(by=['Date Posted'],ascending=False)
link_jobs_unq.to_csv('jobs.csv',index = False)